In [3]:
import nibabel as nib
from nibabel import processing as prc
import numpy as np
import os

In [ ]:
subjectlist = open('/media/corey/4TB-WDBlue/data-thesis/subject_list_full.txt')
sublist = subjectlist.readlines()

for n, sub in enumerate(sublist):
    sub = sub.strip()
    print(n, sub)
    wmparc = nib.load('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/MNINonLinear/wmparc.nii.gz' %sub)
    wmparc_data = wmparc.get_fdata()
    affine = wmparc.affine
    if os.path.exists('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/rfMRI_REST1_PA/'):
        epi_ts = nib.load('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/MNINonLinear/Results/rfMRI_REST1_PA/rfMRI_REST1_PA.nii.gz' %sub)
        epi_ts_data = epi_ts.get_fdata()
        epi_mask = nib.load('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/MNINonLinear/Results/rfMRI_REST1_PA/rfMRI_REST1_PA_finalmask.nii.gz' %sub)
    if os.path.exists('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/rfMRI_REST2_PA/'):
        epi_ts = nib.load('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/MNINonLinear/Results/rfMRI_REST2_PA/rfMRI_REST2_PA' %sub)
        epi_ts_data = epi_ts.get_fdata()
        epi_mask = nib.load('/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/MNINonLinear/Results/rfMRI_REST2_PA/rfMRI_REST2_PA_finalmask.nii.gz' %sub)
    
#Generate mask and resample to MNI152 2mm space (to match epi time series)
    wm_mask = np.zeros_like(wmparc_data)

    for x in range(0,wm_mask.shape[0]-1):
        for y in range(0,wm_mask.shape[1]-1):
            for z in range(0,wm_mask.shape[2]-1):
                if wmparc_data[x,y,z] >= 3000:
                    wm_mask[x,y,z] = 1
    wm_mask_nifti = nib.Nifti1Image(wm_mask,affine)

    resampled_wm_mask = prc.resample_from_to(wm_mask_nifti,epi_mask,order=0)
    nib.save(resampled_wm_mask,'/media/corey/My_Passport/DATA/fMRI/ext_data/EP_FunctionalPreprocessing/%s_01_MR/resampled_wm_mask.nii.gz' %sub)